In [ ]:
import imageio
import io
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
from scipy.special import softmax
import ternary
import time
from tqdm import tqdm
import utils

%matplotlib inline

In [ ]:
if "maler" in str(Path(".").resolve()):
    absolute_path = Path("C:/Users/maler/Federico/Lavoro/ZIB/experiments_results") 
elif "luziehel" in str(Path(".").resolve()):
    absolute_path = Path("C:/Users/luziehel/Code/experiments_results")

series_dir = absolute_path.joinpath("2_opt-h_luzie-gamma_delta_expl")
series_dir = absolute_path.joinpath("2_opt-h_luzie-alpha_beta_gamma_delta_expl")
series_dir = absolute_path.joinpath("2_opt-h_luzie-alpha_beta_gamma_delta_expl-0.0001t")

experiment_runs_folders = [folder for folder in series_dir.iterdir()]

# Single Random Experiment Plot

In [ ]:
folder = np.random.choice(experiment_runs_folders)
metrics_results = utils.load_metrics(folder)

print(folder)

# This is used to linearize the plots by sampling on every x time_steps
x = 1

state_metric = metrics_results['StateMetric'][::x]
n_agents = state_metric.shape[1]
n_options = state_metric.shape[2]

sigma = 0.2
softmax_options = softmax(sigma*state_metric, axis=2)

utils.plot_agents_option(state_metric)

In [ ]:
utils.plot_agents_option(softmax_options)

# Save Plot

In [ ]:
save_path = Path(f"Plots/wavelike-{folder.name}.png")
utils.plot_agents_option(state_metric, save_path=save_path)

# Defined Experiment Plot

In [ ]:
folder = series_dir.joinpath("0.8alpha_-0.4beta_-1.6gamma_-0.8delta")
metrics_results = utils.load_metrics(os.path.join(experiment_folder, folder))

# This is used to linearize the plots by sampling on every x time_steps
x = 1

state_metric = metrics_results['StateMetric'][::x]
n_agents = state_metric.shape[1]
n_options = state_metric.shape[2]

sigma = 0.2
softmax_options = softmax(sigma*state_metric, axis=2)

utils.plot_agents_option(state_metric)

# GIF Generation

In [ ]:
save_path = Path(f"Plots/wavelike-{folder.name}.gif")
utils.generate_gif(save_path=save_path, state_metric=state_metric)

# Series Plot

In [ ]:
for folder in experiment_runs_folders:
    gamma, delta = [param[:3] for param in folder.name.split("_")]
    metrics_results = utils.load_metrics(folder)

In [ ]:
agreement_folders = []
for folder in tqdm(experiment_runs_folders):
    if folder.is_dir():
        metrics_results = utils.load_metrics(folder)
        if np.any(np.all(metrics_results["AgreementMetric"] == 1, axis=1)):
            agreement_folders.append(folder.name)

print("\n", len(agreement_folders))

In [ ]:
stable_agreement_folders = []
for folder in tqdm(experiment_runs_folders):
    if folder.is_dir():
        metrics_results = utils.load_metrics(folder)
        if np.any(np.all(metrics_results["AgreementMetric"] == 1, axis=1)):
            start_agreement = np.min(np.argwhere(np.all(metrics_results["AgreementMetric"] == 1, axis=1)))
            if np.all(np.all(metrics_results["AgreementMetric"][start_agreement:] == 1, axis=1)):
                stable_agreement_folders.append(folder.name)

print("\n", len(stable_agreement_folders))

In [ ]:
stable_disagreement_folders = []
check = []
alternative = []
last_check = []
for folder in tqdm(experiment_runs_folders):
    if folder.is_dir():
        metrics_results = utils.load_metrics(folder)
        agreement_metric = metrics_results["AgreementMetric"][::50]
        flag = False
        for i in range(agreement_metric.shape[0] - 1):
            if np.all(agreement_metric[i] == agreement_metric[i+1]):
                flag = True
                if np.mean(np.all(agreement_metric[i+1:] == agreement_metric[i], axis=1)) > 0.95:
                    stable_disagreement_folders.append(folder.name)
                    break
                elif np.mean(np.all(agreement_metric[i+1:] == agreement_metric[i], axis=1)) < 0.02:
                    check.append(folder.name)
                    break
                else:
                    alternative.append(folder.name)
                    break
        if not flag:
            last_check.append(folder.name)
                

print("\n", len(stable_disagreement_folders))

In [ ]:
print(len(stable_disagreement_folders), len(check), len(alternative), len(last_check))

In [ ]:
consensus_folders = []
for folder in tqdm(experiment_runs_folders):
    if folder.is_dir():
        metrics_results = utils.load_metrics(folder)
        if np.any(metrics_results["ConsensusMetric"] == 1):
            consensus_folders.append(folder.name)

print(consensus_folders)